In [28]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


In [29]:
#Khởi tạo mediapie
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Hàm tính khoảng cách

In [30]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    '''
    Calculate angle between 3 points in 3D space using dot product.
    '''
    A = np.array(pointA)
    B = np.array(pointB)
    C = np.array(pointC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle


## 3. Lỗi gối quá hẹp 
Vì tỉ lệ giữa gối và hông khi lên, xuống trong 1 chu kỳ là khác nhau, ta cần xác định giai đoạn của độngtacs trước khi xét


In [31]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [32]:
#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()

In [33]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

Xem data chuẩn

In [34]:
#Làm sạch dữ liệu dataframe trước khi thêm dữ liệu mới
df = pd.DataFrame(columns=["video", "stage", "shoulder_width", "feet_width"])

In [52]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def process_frame(Video_folder, Video_name):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính khoảng cách
            knee_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách đầu gối, chân
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                predicted_class = trained_model.predict(X)[0]
                prediction_probability = trained_model.predict_proba(X)[0]

                current_stage ="middle"
                if prediction_probability[prediction_probability.argmax()] >= 0.7: # Nếu xác suất dự đoán lớn hơn 0.7 thì mới cập nhật trạng thái
                    if predicted_class == 0:
                        current_stage = "down"
                    elif current_stage == "middle" and predicted_class == 1:
                        current_stage = "up"
                else:
                    current_stage = "middle"

                # Khoảng cách giữa 2 đầu gối 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                knee_width = calculate_distance(left_knee, right_knee)

                # Khoảng cách giữa 2 chân
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)


                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee distance
                cv2.putText(image, "KNEE", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            df.loc[len(df)] = [Video_name, current_stage, knee_width, feet_width]

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# process_frame(video_folder, video_files)



In [62]:
import os

# Đường dẫn thư mục chứa video
video_folder = "Data/Train/Correct"
# video_folder = "Data/Train/Dau_goi_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

df = pd.DataFrame(columns=["video", "stage", "knee_width", "feet_width"])

for video in video_files:
    process_frame(video_folder, video)

In [63]:
df 

,video,stage,knee_width,feet_width
0,20250228_080937000_iOS.mp4,up,0.071030,0.068971
1,20250228_080937000_iOS.mp4,up,0.071130,0.069843
2,20250228_080937000_iOS.mp4,up,0.071192,0.070256
3,20250228_080937000_iOS.mp4,up,0.071299,0.070583
4,20250228_080937000_iOS.mp4,up,0.071467,0.070736
...,...,...,...,...
1919,VID_20250228_152348.mp4,up,0.061135,0.066572
1920,VID_20250228_152348.mp4,up,0.060966,0.066693
1921,VID_20250228_152348.mp4,up,0.060695,0.066725
1922,VID_20250228_152348.mp4,up,0.060736,0.066704


In [64]:
#Tính toán tỉ lệ giữa vai và chân
df["ratio"] = df["knee_width"] / df["feet_width"]
df

,video,stage,knee_width,feet_width,ratio
0,20250228_080937000_iOS.mp4,up,0.071030,0.068971,1.029853
1,20250228_080937000_iOS.mp4,up,0.071130,0.069843,1.018427
2,20250228_080937000_iOS.mp4,up,0.071192,0.070256,1.013321
3,20250228_080937000_iOS.mp4,up,0.071299,0.070583,1.010144
4,20250228_080937000_iOS.mp4,up,0.071467,0.070736,1.010341
...,...,...,...,...,...
1919,VID_20250228_152348.mp4,up,0.061135,0.066572,0.918330
1920,VID_20250228_152348.mp4,up,0.060966,0.066693,0.914139
1921,VID_20250228_152348.mp4,up,0.060695,0.066725,0.909634
1922,VID_20250228_152348.mp4,up,0.060736,0.066704,0.910521


In [65]:
#Phân tích bộ data
df_group = df.groupby("stage").describe()
# Lọc chỉ các cột có mean, min, max
df_filtered = df_group.loc[:, (slice(None), ['count', 'mean', 'min', 'max'])]

# Hiển thị kết quả
print(df_filtered)

       knee_width                               feet_width            \
            count      mean       min       max      count      mean   
stage                                                                  
down        347.0  0.124357  0.086977  0.171286      347.0  0.072566   
middle      869.0  0.112759  0.061800  0.170036      869.0  0.075316   
up          708.0  0.073679  0.057333  0.115880      708.0  0.071475   

                            ratio                                
             min       max  count      mean       min       max  
stage                                                            
down    0.058701  0.093726  347.0  1.702238  1.421656  2.098558  
middle  0.059299  0.095393  869.0  1.482528  0.956121  2.132808  
up      0.055971  0.086649  708.0  1.033327  0.883585  1.447330  


In [57]:
df_filtered = df_group[df_group["ratio"] < 0.88]
print(df_filtered)


ValueError: cannot join with no overlapping index names

# Kết luận:
Với ratio = knee/ feet <br>
Khi stage = up: ratio < 0.88: gối hẹp <br>
Khi stage = middle: ratio < 0.96: gối hẹp <br>
Khi stage = down: ratio < 1.42: gối hẹp <br>
